In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

# Config

In [20]:
data_dir = "data"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

# Utilisation de ImageDataGenerator pour prétraiter les images

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Génération des données depuis les dossiers

In [22]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 27032 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [23]:
print(f"Classes : {train_generator.class_indices}")
class_names = list(train_generator.class_indices.keys())

Classes : {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


# 2. Chargement du modèle pré-entraîné VGG16

In [24]:
# Adapter les données en niveaux de gris pour être compatibles avec VGG16
input_layer = Input(shape=(48, 48, 1))
converted_layer = Conv2D(3, (3, 3), padding='same', activation='relu')(input_layer)

# Charger VGG16 pré-entraîné
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=converted_layer)

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Ajouter des couches personnalisées pour le fine-tuning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(class_names), activation='softmax')(x)

# Définir le modèle final
model = Model(inputs=input_layer, outputs=output)

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


ValueError: Layer count mismatch when loading weights from file. Model expected 14 layers, found 13 saved layers.

# 3. Entraînement du modèle

In [ ]:
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=test_generator,
    verbose=1,
)

347/845 ━━━━━━━━━━━━━━━━━━━━ 52:28 6s/step - accuracy: 0.2565 - loss: 1.7807

# 4. Évaluation du modèle

In [9]:
loss, accuracy = model.evaluate(test_generator, verbose=2)
print(f"\nPrécision sur le jeu de test : {accuracy * 100:.2f}%")

KeyboardInterrupt: 

# 5. Visualisation des courbes d'entraînement

In [ ]:
plt.figure(figsize=(12, 4))

# Courbe de perte
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Perte Entraînement')
plt.plot(history.history['val_loss'], label='Perte Validation')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()
plt.title('Évolution de la perte')

# Courbe de précision
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Précision Entraînement')
plt.plot(history.history['val_accuracy'], label='Précision Validation')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()
plt.title('Évolution de la précision')

plt.show()

# 6. Matrice de confusion et rapport de classification

In [ ]:
y_true = test_generator.classes
test_generator.reset()
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Prédictions')
plt.ylabel('Vérités')
plt.title('Matrice de Confusion')
plt.show()

print("\nRapport de classification :\n")
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# 7. Visualisation des prédictions

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(16):
    img, label = test_generator[i]
    pred_label = class_names[np.argmax(y_pred_classes[i])]
    true_label = class_names[label[i].argmax()]
    color = 'green' if pred_label == true_label else 'red'
    plt.subplot(4, 4, i + 1)
    plt.imshow(img[0].astype('uint8'))
    plt.title(f"{pred_label} ({true_label})", color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()